In [1]:
#are you connected to the campus vpn full tunneling? if not, forget about it
import cobra
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cometspy as c
import os

In [2]:
E_WT_infected = cobra.io.read_sbml_model("../models/iJO1366_plasmid_phage.xml")
S0 = cobra.io.read_sbml_model("../models/STM_v1_0_S0.xml")
M0 = cobra.io.read_sbml_model("../models/jmc_AM1_KO_renamed.xml")

E0_infected = E_WT_infected.copy()
E0_infected.genes.b3939.knock_out()
#E0_infected.reactions.plasmid_F.reaction = '1.0487663835663 ala__L_c + 0.375189220845313 arg__L_c + 0.252042985698166 asn__L_c + 0.23012847099302 asp__L_c + 0.0617971010563441 cys__L_c + 31.4390293663471 atp_c + 0.000310535123271839 dctp_c + 0.000310535123271839 dgtp_c + 0.000307719504895013 datp_c + 0.000307719504895013 dttp_c + 0.239453533629087 gln__L_c + 0.166879939481303 glu__L_c + 0.550578201069248 gly_c + 31.4387216468422 h2o_c + 0.109480365927243 his__L_c + 0.303202311009479 ile__L_c + 0.955403496783467 leu__L_c + 0.318730982418103 lys__L_c + 0.425808509511056 met__L_c + 0.479268785743603 phe__L_c + 0.222207077389324 pro__L_c + 0.528621980342174 ser__L_c + 0.388434090062005 thr__L_c + 0.221056983156149 trp__L_c + 0.144291651462158 tyr__L_c + 0.838338349373724 val__L_c --> 31.4387216468422 adp_c + 31.4387216468422 h_c + 31.4387216468422 pi_c + 0.00123650405186351 ppi_c'
#E0_infected.reactions.phage_M13.reaction = '0.898142036193534 ala__L_c + 0.136473533481805 arg__L_c + 0.138976716414339 asn__L_c + 0.222182517091801 asp__L_c + 0.0345439244689821 cys__L_c + 27.9053031027283 atp_c + 0.0523065105582499 dctp_c + 0.0523065105582499 dgtp_c + 0.0759966338317605 dttp_c + 0.0759966338317605 datp_c + 0.250218165936192 gln__L_c + 0.24080619810986 glu__L_c + 0.454778275182947 gly_c + 27.8293064688965 h2o_c + 0.0338430332478723 his__L_c + 0.359857578381222 ile__L_c + 0.665346023467785 leu__L_c + 0.635307828277366 lys__L_c + 0.228090028812583 met__L_c + 0.327816836844775 phe__L_c + 0.304987808500056 pro__L_c + 0.631102480950707 ser__L_c + 0.358856305208208 thr__L_c + 0.0578735894002076 trp__L_c + 0.27374808550202 tyr__L_c + 0.70439567721533 val__L_c --> 27.8293064688965 adp_c + 27.8293064688965 h_c + 27.8293064688965 pi_c + 0.2565662378531 ppi_c'

# ESM

In [3]:
#phage bound only
E0_infected.objective = E0_infected.reactions.phage_M13

E_infected = c.model(E0_infected)
S = c.model(S0)
M = c.model(M0)
E_infected.open_exchanges()
S.open_exchanges()
M.open_exchanges()

E_infected.initial_pop = [0, 0, 1.e-8]
S.initial_pop = [0, 0, 1.e-8]
M.initial_pop = [0, 0, 1.e-8]
E_infected.obj_style = "MAXIMIZE_OBJECTIVE_FLUX"
S.obj_style =  "MAXIMIZE_OBJECTIVE_FLUX"
M.obj_style =  "MAXIMIZE_OBJECTIVE_FLUX"

l = c.layout([E_infected,S,M])
base_nutrients = ["ca2_e", "cl_e", "cobalt2_e", "cu2_e","fe2_e", "fe3_e", "k_e","mg2_e",
              "mn2_e", "mobd_e", "ni2_e", "o2_e", "pi_e", "so4_e", "zn2_e"]
for nutrient in base_nutrients:
    l.set_specific_metabolite(nutrient, 1000)
l.set_specific_metabolite("lcts_e", 0.000278)
l.set_specific_metabolite("mea_e", 0.0015)

p = c.params()
p.set_param("defaultKm", 0.00001) # M 
p.set_param("defaultVmax", 10) #mmol/gDw/hr
p.set_param("maxCycles", 200)
p.set_param("timeStep", 1)
p.set_param('writeMediaLog', True)
p.set_param('MediaLogRate', 1)
p.set_param('writeFluxLog', True)
p.set_param('FluxLogRate', 1)
p.set_param("totalBiomassLogRate", 1)

sim = c.comets(l, p)
sim.run()

ESM_bound_phage = max(sim.fluxes_by_species["iJO1366"]["phage_M13"])


Running COMETS simulation ...

Debug Here ...
Done!


In [4]:
#plasmid bound only
E0_infected.objective = E0_infected.reactions.plasmid_F

E_infected = c.model(E0_infected)
S = c.model(S0)
M = c.model(M0)
E_infected.open_exchanges()
S.open_exchanges()
M.open_exchanges()

E_infected.initial_pop = [0, 0, 1.e-8]
S.initial_pop = [0, 0, 1.e-8]
M.initial_pop = [0, 0, 1.e-8]
E_infected.obj_style = "MAXIMIZE_OBJECTIVE_FLUX"
S.obj_style =  "MAXIMIZE_OBJECTIVE_FLUX"
M.obj_style =  "MAXIMIZE_OBJECTIVE_FLUX"

l = c.layout([E_infected,S,M])
base_nutrients = ["ca2_e", "cl_e", "cobalt2_e", "cu2_e","fe2_e", "fe3_e", "k_e","mg2_e",
              "mn2_e", "mobd_e", "ni2_e", "o2_e", "pi_e", "so4_e", "zn2_e"]
for nutrient in base_nutrients:
    l.set_specific_metabolite(nutrient, 1000)
l.set_specific_metabolite("lcts_e", 0.000278)
l.set_specific_metabolite("mea_e", 0.0015)

p = c.params()
p.set_param("defaultKm", 0.00001) # M 
p.set_param("defaultVmax", 10) #mmol/gDw/hr
p.set_param("maxCycles", 200)
p.set_param("timeStep", 1)
p.set_param('writeMediaLog', True)
p.set_param('MediaLogRate', 1)
p.set_param('writeFluxLog', True)
p.set_param('FluxLogRate', 1)
p.set_param("totalBiomassLogRate", 1)

sim = c.comets(l, p)
sim.run()

ESM_bound_plasmid = max(sim.fluxes_by_species["iJO1366"]["plasmid_F"])


Running COMETS simulation ...

Debug Here ...
Done!


In [5]:
#phage and plasmid bound
E0_infected.objective = E0_infected.reactions.phage_M13
E0_infected.reactions.plasmid_F.lower_bound = 0.3

E_infected = c.model(E0_infected)
S = c.model(S0)
M = c.model(M0)
E_infected.open_exchanges()
S.open_exchanges()
M.open_exchanges()

E_infected.initial_pop = [0, 0, 1.e-8]
S.initial_pop = [0, 0, 1.e-8]
M.initial_pop = [0, 0, 1.e-8]
E_infected.obj_style = "MAXIMIZE_OBJECTIVE_FLUX"
S.obj_style =  "MAXIMIZE_OBJECTIVE_FLUX"
M.obj_style =  "MAXIMIZE_OBJECTIVE_FLUX"

l = c.layout([E_infected,S,M])
base_nutrients = ["ca2_e", "cl_e", "cobalt2_e", "cu2_e","fe2_e", "fe3_e", "k_e","mg2_e",
              "mn2_e", "mobd_e", "ni2_e", "o2_e", "pi_e", "so4_e", "zn2_e"]
for nutrient in base_nutrients:
    l.set_specific_metabolite(nutrient, 1000)
l.set_specific_metabolite("lcts_e", 0.000278)
l.set_specific_metabolite("mea_e", 0.0015)

p = c.params()
p.set_param("defaultKm", 0.00001) # M 
p.set_param("defaultVmax", 10) #mmol/gDw/hr
p.set_param("maxCycles", 200)
p.set_param("timeStep", 1)
p.set_param('writeMediaLog', True)
p.set_param('MediaLogRate', 1)
p.set_param('writeFluxLog', True)
p.set_param('FluxLogRate', 1)
p.set_param("totalBiomassLogRate", 1)

sim = c.comets(l, p)
sim.run()

ESM_bound_phageplasmid = max(sim.fluxes_by_species["iJO1366"]["phage_M13"])


Running COMETS simulation ...

Debug Here ...
Done!


# ES

In [6]:
#phage only
E0_infected.reactions.plasmid_F.lower_bound = 0.0
E0_infected.objective = E0_infected.reactions.phage_M13

E_infected = c.model(E0_infected)
S = c.model(S0)
E_infected.open_exchanges()
S.open_exchanges()

E_infected.initial_pop = [0, 0, 1.e-8]
S.initial_pop = [0, 0, 1.e-8]
E_infected.obj_style = "MAXIMIZE_OBJECTIVE_FLUX"
S.obj_style =  "MAXIMIZE_OBJECTIVE_FLUX"

l = c.layout([E_infected,S])
base_nutrients = ["ca2_e", "cl_e", "cobalt2_e", "cu2_e","fe2_e", "fe3_e", "k_e","mg2_e",
              "mn2_e", "mobd_e", "ni2_e", "o2_e", "nh4_e","pi_e", "so4_e", "zn2_e"]
for nutrient in base_nutrients:
    l.set_specific_metabolite(nutrient, 1000)
l.set_specific_metabolite("lcts_e", 0.000278)

p = c.params()
p.set_param("defaultKm", 0.00001) # M 
p.set_param("defaultVmax", 10) #mmol/gDw/hr
p.set_param("maxCycles", 200)
p.set_param("timeStep", 1)
p.set_param('writeMediaLog', True)
p.set_param('MediaLogRate', 1)
p.set_param('writeFluxLog', True)
p.set_param('FluxLogRate', 1)
p.set_param("totalBiomassLogRate", 1)

sim = c.comets(l, p)
sim.run()

ES_bound_phage = max(sim.fluxes_by_species["iJO1366"]["phage_M13"])


Running COMETS simulation ...

Debug Here ...
Done!


In [7]:
#plasmid only
E0_infected.objective = E0_infected.reactions.plasmid_F

E_infected = c.model(E0_infected)
S = c.model(S0)
E_infected.open_exchanges()
S.open_exchanges()

E_infected.initial_pop = [0, 0, 1.e-8]
S.initial_pop = [0, 0, 1.e-8]
E_infected.obj_style = "MAXIMIZE_OBJECTIVE_FLUX"
S.obj_style =  "MAXIMIZE_OBJECTIVE_FLUX"

l = c.layout([E_infected,S])
base_nutrients = ["ca2_e", "cl_e", "cobalt2_e", "cu2_e","fe2_e", "fe3_e", "k_e","mg2_e",
              "mn2_e", "mobd_e", "ni2_e", "o2_e", "nh4_e","pi_e", "so4_e", "zn2_e"]
for nutrient in base_nutrients:
    l.set_specific_metabolite(nutrient, 1000)
l.set_specific_metabolite("lcts_e", 0.000278)

p = c.params()
p.set_param("defaultKm", 0.00001) # M 
p.set_param("defaultVmax", 10) #mmol/gDw/hr
p.set_param("maxCycles", 200)
p.set_param("timeStep", 1)
p.set_param('writeMediaLog', True)
p.set_param('MediaLogRate', 1)
p.set_param('writeFluxLog', True)
p.set_param('FluxLogRate', 1)
p.set_param("totalBiomassLogRate", 1)

sim = c.comets(l, p)
sim.run()

ES_bound_plasmid = max(sim.fluxes_by_species["iJO1366"]["plasmid_F"])


Running COMETS simulation ...

Debug Here ...
Done!


In [8]:
#phage and plasmid
E0_infected.objective = E0_infected.reactions.phage_M13
E0_infected.reactions.plasmid_F.lower_bound = 0.9

E_infected = c.model(E0_infected)
S = c.model(S0)
E_infected.open_exchanges()
S.open_exchanges()

E_infected.initial_pop = [0, 0, 1.e-8]
S.initial_pop = [0, 0, 1.e-8]
E_infected.obj_style = "MAXIMIZE_OBJECTIVE_FLUX"
S.obj_style =  "MAXIMIZE_OBJECTIVE_FLUX"

l = c.layout([E_infected,S])
base_nutrients = ["ca2_e", "cl_e", "cobalt2_e", "cu2_e","fe2_e", "fe3_e", "k_e","mg2_e",
              "mn2_e", "mobd_e", "ni2_e", "o2_e", "nh4_e","pi_e", "so4_e", "zn2_e"]
for nutrient in base_nutrients:
    l.set_specific_metabolite(nutrient, 1000)
l.set_specific_metabolite("lcts_e", 0.000278)

p = c.params()
p.set_param("defaultKm", 0.00001) # M 
p.set_param("defaultVmax", 10) #mmol/gDw/hr
p.set_param("maxCycles", 200)
p.set_param("timeStep", 1)
p.set_param('writeMediaLog', True)
p.set_param('MediaLogRate', 1)
p.set_param('writeFluxLog', True)
p.set_param('FluxLogRate', 1)
p.set_param("totalBiomassLogRate", 1)

sim = c.comets(l, p)
sim.run()

ES_bound_phageplasmid = max(sim.fluxes_by_species["iJO1366"]["phage_M13"])


Running COMETS simulation ...

Debug Here ...
Done!


# E monoculture

In [9]:
#phage only
E0_infected.reactions.plasmid_F.lower_bound = 0.0
E0_infected.objective = E0_infected.reactions.phage_M13

E_infected = c.model(E0_infected)
E_infected.open_exchanges()

E_infected.initial_pop = [0, 0, 1.e-8]
E_infected.obj_style = "MAXIMIZE_OBJECTIVE_FLUX"

l = c.layout([E_infected])
base_nutrients = ["ca2_e", "cl_e", "cobalt2_e", "cu2_e","fe2_e", "fe3_e", "k_e","mg2_e",
              "mn2_e", "mobd_e", "ni2_e", "o2_e", "nh4_e","pi_e", "so4_e", "zn2_e"]
for nutrient in base_nutrients:
    l.set_specific_metabolite(nutrient, 1000)
l.set_specific_metabolite("lcts_e", 0.000278)
l.set_specific_metabolite("met__L_e", 10)

p = c.params()
p.set_param("defaultKm", 0.00001) # M 
p.set_param("defaultVmax", 10) #mmol/gDw/hr
p.set_param("maxCycles", 200)
p.set_param("timeStep", 1)
p.set_param('writeMediaLog', True)
p.set_param('MediaLogRate', 1)
p.set_param('writeFluxLog', True)
p.set_param('FluxLogRate', 1)
p.set_param("totalBiomassLogRate", 1)

sim = c.comets(l, p)
sim.run()

E_bound_phage = max(sim.fluxes_by_species["iJO1366"]["phage_M13"])


Running COMETS simulation ...

Debug Here ...
Done!


In [10]:
#plasmid only
E0_infected.objective = E0_infected.reactions.plasmid_F

E_infected = c.model(E0_infected)
E_infected.open_exchanges()

E_infected.initial_pop = [0, 0, 1.e-8]
E_infected.obj_style = "MAXIMIZE_OBJECTIVE_FLUX"

l = c.layout([E_infected])
base_nutrients = ["ca2_e", "cl_e", "cobalt2_e", "cu2_e","fe2_e", "fe3_e", "k_e","mg2_e",
              "mn2_e", "mobd_e", "ni2_e", "o2_e", "nh4_e","pi_e", "so4_e", "zn2_e"]
for nutrient in base_nutrients:
    l.set_specific_metabolite(nutrient, 1000)
l.set_specific_metabolite("lcts_e", 0.000278)
l.set_specific_metabolite("met__L_e", 10)

p = c.params()
p.set_param("defaultKm", 0.00001) # M 
p.set_param("defaultVmax", 10) #mmol/gDw/hr
p.set_param("maxCycles", 200)
p.set_param("timeStep", 1)
p.set_param('writeMediaLog', True)
p.set_param('MediaLogRate', 1)
p.set_param('writeFluxLog', True)
p.set_param('FluxLogRate', 1)
p.set_param("totalBiomassLogRate", 1)

sim = c.comets(l, p)
sim.run()

E_bound_plasmid = max(sim.fluxes_by_species["iJO1366"]["plasmid_F"])


Running COMETS simulation ...

Debug Here ...
Done!


In [11]:
#phage with plasmid
E0_infected.reactions.plasmid_F.lower_bound = 0.9
E0_infected.objective = E0_infected.reactions.phage_M13

E_infected = c.model(E0_infected)
E_infected.open_exchanges()

E_infected.initial_pop = [0, 0, 1.e-8]
E_infected.obj_style = "MAXIMIZE_OBJECTIVE_FLUX"

l = c.layout([E_infected])
base_nutrients = ["ca2_e", "cl_e", "cobalt2_e", "cu2_e","fe2_e", "fe3_e", "k_e","mg2_e",
              "mn2_e", "mobd_e", "ni2_e", "o2_e", "nh4_e","pi_e", "so4_e", "zn2_e"]
for nutrient in base_nutrients:
    l.set_specific_metabolite(nutrient, 1000)
l.set_specific_metabolite("lcts_e", 0.000278)
l.set_specific_metabolite("met__L_e", 10)

p = c.params()
p.set_param("defaultKm", 0.00001) # M 
p.set_param("defaultVmax", 10) #mmol/gDw/hr
p.set_param("maxCycles", 200)
p.set_param("timeStep", 1)
p.set_param('writeMediaLog', True)
p.set_param('MediaLogRate', 1)
p.set_param('writeFluxLog', True)
p.set_param('FluxLogRate', 1)
p.set_param("totalBiomassLogRate", 1)

sim = c.comets(l, p)
sim.run()

E_bound_phageplasmid = max(sim.fluxes_by_species["iJO1366"]["phage_M13"])


Running COMETS simulation ...

Debug Here ...
Done!


# EM

In [12]:
#phage only
E0_infected.reactions.plasmid_F.lower_bound = 0.0
E0_infected.objective = E0_infected.reactions.phage_M13

E_infected = c.model(E0_infected)
M = c.model(M0)
E_infected.open_exchanges()
M.open_exchanges()

E_infected.initial_pop = [0, 0, 1.e-8]
M.initial_pop = [0, 0, 1.e-8]
E_infected.obj_style = "MAXIMIZE_OBJECTIVE_FLUX"
M.obj_style =  "MAXIMIZE_OBJECTIVE_FLUX"

l = c.layout([E_infected,M])
base_nutrients = ["ca2_e", "cl_e", "cobalt2_e", "cu2_e","fe2_e", "fe3_e", "k_e","mg2_e",
              "mn2_e", "mobd_e", "ni2_e", "o2_e", "pi_e", "so4_e", "zn2_e"]
for nutrient in base_nutrients:
    l.set_specific_metabolite(nutrient, 1000)
l.set_specific_metabolite("lcts_e", 0.000278)
l.set_specific_metabolite("mea_e", 0.0015)
l.set_specific_metabolite("met__L_e", 10)

p = c.params()
p.set_param("defaultKm", 0.00001) # M 
p.set_param("defaultVmax", 10) #mmol/gDw/hr
p.set_param("maxCycles", 200)
p.set_param("timeStep", 1)
p.set_param('writeMediaLog', True)
p.set_param('MediaLogRate', 1)
p.set_param('writeFluxLog', True)
p.set_param('FluxLogRate', 1)
p.set_param("totalBiomassLogRate", 1)

sim = c.comets(l, p)
sim.run()

EM_bound_phage = max(sim.fluxes_by_species["iJO1366"]["phage_M13"])


Running COMETS simulation ...

Debug Here ...
Done!


In [13]:
#plasmid only
E0_infected.objective = E0_infected.reactions.plasmid_F

E_infected = c.model(E0_infected)
M = c.model(M0)
E_infected.open_exchanges()
M.open_exchanges()

E_infected.initial_pop = [0, 0, 1.e-8]
M.initial_pop = [0, 0, 1.e-8]
E_infected.obj_style = "MAXIMIZE_OBJECTIVE_FLUX"
M.obj_style =  "MAXIMIZE_OBJECTIVE_FLUX"

l = c.layout([E_infected,M])
base_nutrients = ["ca2_e", "cl_e", "cobalt2_e", "cu2_e","fe2_e", "fe3_e", "k_e","mg2_e",
              "mn2_e", "mobd_e", "ni2_e", "o2_e", "pi_e", "so4_e", "zn2_e"]
for nutrient in base_nutrients:
    l.set_specific_metabolite(nutrient, 1000)
l.set_specific_metabolite("lcts_e", 0.000278)
l.set_specific_metabolite("mea_e", 0.0015)
l.set_specific_metabolite("met__L_e", 10)

p = c.params()
p.set_param("defaultKm", 0.00001) # M 
p.set_param("defaultVmax", 10) #mmol/gDw/hr
p.set_param("maxCycles", 200)
p.set_param("timeStep", 1)
p.set_param('writeMediaLog', True)
p.set_param('MediaLogRate', 1)
p.set_param('writeFluxLog', True)
p.set_param('FluxLogRate', 1)
p.set_param("totalBiomassLogRate", 1)

sim = c.comets(l, p)
sim.run()

EM_bound_plasmid = max(sim.fluxes_by_species["iJO1366"]["plasmid_F"])


Running COMETS simulation ...

Debug Here ...
Done!


In [14]:
#phage with plasmid
E0_infected.reactions.plasmid_F.lower_bound = 0.9
E0_infected.objective = E0_infected.reactions.phage_M13

E_infected = c.model(E0_infected)
M = c.model(M0)
E_infected.open_exchanges()
M.open_exchanges()

E_infected.initial_pop = [0, 0, 1.e-8]
M.initial_pop = [0, 0, 1.e-8]
E_infected.obj_style = "MAXIMIZE_OBJECTIVE_FLUX"
M.obj_style =  "MAXIMIZE_OBJECTIVE_FLUX"

l = c.layout([E_infected,M])
base_nutrients = ["ca2_e", "cl_e", "cobalt2_e", "cu2_e","fe2_e", "fe3_e", "k_e","mg2_e",
              "mn2_e", "mobd_e", "ni2_e", "o2_e", "pi_e", "so4_e", "zn2_e"]
for nutrient in base_nutrients:
    l.set_specific_metabolite(nutrient, 1000)
l.set_specific_metabolite("lcts_e", 0.000278)
l.set_specific_metabolite("mea_e", 0.0015)
l.set_specific_metabolite("met__L_e", 10)

p = c.params()
p.set_param("defaultKm", 0.00001) # M 
p.set_param("defaultVmax", 10) #mmol/gDw/hr
p.set_param("maxCycles", 200)
p.set_param("timeStep", 1)
p.set_param('writeMediaLog', True)
p.set_param('MediaLogRate', 1)
p.set_param('writeFluxLog', True)
p.set_param('FluxLogRate', 1)
p.set_param("totalBiomassLogRate", 1)

sim = c.comets(l, p)
sim.run()

EM_bound_phageplasmid = max(sim.fluxes_by_species["iJO1366"]["phage_M13"])


Running COMETS simulation ...

Debug Here ...
Done!


# Print the final values for easy future use 

In [15]:
print(E_bound_phage)
print(E_bound_plasmid)
print(E_bound_phageplasmid)

1.0408699846
1.0227541037
0.12492842765


In [16]:
print(E_bound_phage)
print(ES_bound_phage)
print(EM_bound_phage)
print(ESM_bound_phage)

1.0408699846
0.56350519265
0.5249955979
0.53267028898


In [17]:
print(E_bound_plasmid)
print(ES_bound_plasmid)
print(EM_bound_plasmid)
print(ESM_bound_plasmid)

1.0227541037
0.65154948653
0.51137705186
0.60563141258


In [18]:
print(E_bound_phageplasmid)
print(ES_bound_phageplasmid)
print(EM_bound_phageplasmid)
print(ESM_bound_phageplasmid)

0.12492842765
0.0
0.1249282966
0.0
